In [3]:
import GetOldTweets3 as got

In [4]:
import sklearn.decomposition

ModuleNotFoundError: No module named 'sklearn'

In [2]:
twitter_user_name = 'LambdaSchool'
count = 20 # very small for efficiency during testing

In [3]:
#  Create object to execute queries
querySpecs = got.manager.TweetCriteria().setUsername(twitter_user_name).setMaxTweets(count)

In [4]:
retrieved_tweets = got.manager.TweetManager.getTweets(querySpecs)

In [5]:
# type(retrieved_tweets)

In [6]:
test_tweet = retrieved_tweets[1]
# type(test_tweet.urls)
# 
# dir (test_tweet)

In [7]:
def tweet_to_dict(twt):
    """Munges a twt object into a dict, using names of attributes of
    object as keys in dict.
    'favorites' is a count of 'likes'
    'hashtags' is a string that is a space-separated series of hashtags
    'mentions' is a string that is a space-separated series of ats (@s)
    'urls' is a string that is a space-separated series of URLs
    """
    return {'date' : twt.date
            , 'favorites' : twt.favorites
            , 'formatted_date' : twt.formatted_date
            , 'geo' : twt.geo
            , 'hashtags' : twt.hashtags
            , 'id' : twt.id
            , 'mentions' : twt.mentions
            , 'permalink' : twt.permalink
            , 'replies' : twt.replies
            , 'retweets' : twt.retweets
            , 'text' : twt.text
            , 'to' : twt.to
            , 'urls' : twt.urls
            , 'username' : twt.username}   

In [8]:
def munge_date(dt):
    """Munges a datetime.datetime object into a dict, using names of attributes of
    object as keys in dict.
    'day_of_week' is [0-7] with 0 being 'Monday'
    'minute_of_day' is count of minutes from midnight"""
    return {'year' : dt.year 
            , 'month' : dt.month
            , 'day' : dt.day
            , 'day_of_week' : dt.weekday()
            , 'hour' : dt.hour
            , 'minute' : dt.minute
            , 'minute_of_day' : (60 * dt.hour) + dt.minute}
munge_date(test_tweet.date)

{'year': 2020,
 'month': 4,
 'day': 13,
 'day_of_week': 0,
 'hour': 23,
 'minute': 0,
 'minute_of_day': 1380}

In [9]:
def join_dicts(twt):
    return {**tweet_to_dict(twt), **munge_date(twt.date)}

join_dicts(test_tweet)

{'date': datetime.datetime(2020, 4, 13, 23, 0, 16, tzinfo=datetime.timezone.utc),
 'favorites': 20,
 'formatted_date': 'Mon Apr 13 23:00:16 +0000 2020',
 'geo': '',
 'hashtags': '#RemoteLearning #Remote',
 'id': '1249834817878986753',
 'mentions': '',
 'permalink': 'https://twitter.com/LambdaSchool/status/1249834817878986753',
 'replies': 0,
 'retweets': 3,
 'text': 'For large online lectures of 50+ students, how do you make sure everyone has a good experience? Josh Knell, Head of Instruction at Lambda School, will be tackling this question via Zoom next Wednesday. Register: https://bit.ly/3a4BghW #RemoteLearning #Remote',
 'to': None,
 'urls': 'https://bit.ly/3a4BghW',
 'username': 'LambdaSchool',
 'year': 2020,
 'month': 4,
 'day': 13,
 'day_of_week': 0,
 'hour': 23,
 'minute': 0,
 'minute_of_day': 1380}

In [10]:
### Utilities to make operations on lists less cumbersome.

def remove_duplicates_from_list(l):
    """Stupid Python does not have this."""
    return list(set(l))

def mapcar(func, lst):
    """Stupid Python does not have mapcar."""
    return list(map(func, lst))

def flatten_lists_one_layer(ls):
    from functools import reduce
    """Stupid Python does not have a built-in function to 
    join an arbitrary list of lists into one list."""
    return reduce(lambda accumulator, element: accumulator + element, ls, [])

def join_list_of_dicts(dicts):
    """This is good. Thanks, Python!"""
    return {key: val for dict in dicts for key, val in dict.items()}
 

In [11]:
# def list_of_tweets_hashtags(tweet_object):
#     """NAIVE: assumes 'hashtags' is already a space-delimited
#     string of hashtags."""
#     return tweet_object.hashtags.split()

# def list_of_tweets_mentions(tweet_object):
#     """NAIVE: assumes 'mentions' is already a space-delimited
#     string of @phrases."""
#     return tweet_object.mentions.split()

# def list_of_tweets_hashtags_from_set_of_tweets (set_of_tws):
#     return remove_duplicates_from_list(
#             flatten_lists_one_layer(
#                 mapcar(
#                     list_of_tweets_hashtags, set_of_tws)))

# def list_of_tweets_mentions_from_set_of_tweets (set_of_tws):
#     return remove_duplicates_from_list(
#             flatten_lists_one_layer(
#                 mapcar(
#                     list_of_tweets_mentions, set_of_tws)))

# list_of_tweets_mentions_from_set_of_tweets(retrieved_tweets)

In [12]:
import pandas as pd
# import dictionary as dataframe
# drop useless columns

recs = list(map(join_dicts, retrieved_tweets))

index_col_name = 'id'

useless_columns = ['date', 'formatted_date', 'permalink', 'username', 'hour', 'minute', 'geo']

df = pd.DataFrame.from_records(recs, index=index_col_name, exclude=useless_columns)

In [13]:
df.head()

,favorites,hashtags,mentions,replies,retweets,text,to,urls,year,month,day,day_of_week,minute_of_day
id,,,,,,,,,,,,,
1250084165405597702,16,#hired #SeeYourselfAtLambda,@Fidelity @ryanallred @rrherr @Jon_Cody_,0,2,Congrats to Lambda grad Oscar Calzada for gett...,None,,2020,4,14,1,931
1249834817878986753,20,#RemoteLearning #Remote,,0,3,"For large online lectures of 50+ students, how...",None,https://bit.ly/3a4BghW,2020,4,13,0,1380
1249797109382967296,18,#LambdaSchool,@Divvy_HQ,2,2,"One of our 2019 grad Antonio Melendez, spent f...",None,http://bit.ly/39htNvO,2020,4,13,0,1230
1249456259503403013,14,#engineer,,2,0,What do a software #engineer and a bodybuilder...,None,https://bit.ly/2U7AJqW,2020,4,12,6,1316
1249389342663065600,17,,,0,0,Happy Easter everyone These maybe difficult ti...,None,,2020,4,12,6,1050


In [14]:
# given a df
# return a df with new, populated columns from mentions

all_mentions = flatten_lists_one_layer(list(df['mentions'].str.split()))

# def expand_hashtags(tweet_df):
#     """Modify the dataframe to include columns for each hashtag, 
#     like 'one-hot' encoding, but really 'n-hot' encoding."""
#     # get a list of unique values, for the entire df, of 'mentions'
#     for m in all_mentions:
#         tweet_df[m].apply(lambda x: if (x in df['mentions']): 1 else: 0)
#     return tweet_df

# expand_hashtags(df)

In [15]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html/'/'

In [1]:
def get_space_delimited(col_name):
    """Split a string on spaces, and return a list of the substrings."""
    return flatten_lists_one_layer(list(df[col_name].str.split()))

# Create an n-hot encoding set of columns based on hashtags.    
for hashtag in get_space_delimited('hashtags'):
    df.loc[df['hashtags'].str.contains(hashtag), hashtag] = 1
    df[hashtag].fillna(value=0, inplace=True)

# Create an n-hot encoding set of columns based on mentions.    
for mention in get_space_delimited('mentions'):
    df.loc[df['mentions'].str.contains(mention), mention] = 1
    df[mention].fillna(value=0, inplace=True)

    
# for this to work, need to replace None values with empty strings
# df.loc['to'].fillna(value="", inplace=True) 
# df.loc['to']
# for to_val in get_space_delimited('to'):
#     df.loc[df['to'].str.contains(to_val), to_val] = 1
#     df[to_val].fillna(value=0, inplace=True)
    
    
# text                 object
# to                   object
# urls                 object    
    
    
    
# df['#LambdaSchool']



# # If Python had macros, it would work like


# def blow_out_column(col_name):
#     return ("""for newcol in get_space_delimited(\'""" + col_name +"""')
#     df.loc[df['""" + col_name + """\'].str.contains(newcol), newcol] = 1
#     df[newcol].fillna(value=0, inplace=True)""")
    
## sample output
# print(blow_out_column('to'))

## for newcol in get_space_delimited('mentions')
##     df.loc[df['mentions'].str.contains(newcol), newcol] = 1

NameError: name 'flatten_lists_one_layer' is not defined

In [ ]:
df

In [17]:
import basilica

In [ ]:
sentences = [
    "This is a sentence!",
    "This is a similar sentence!",
    "I don't think this sentence is very similar at all...",
]
with basilica.Connection('SLOW_DEMO_KEY') as c:
    embeddings = list(c.embed_sentences(sentences))
# print(embeddings)
len(embeddings)

In [ ]:
test_tweet


In [ ]:
test_tweet.text

In [ ]:
texts = [tweet.text for tweet in retrieved_tweets]

In [ ]:
texts

In [ ]:
with basilica.Connection('SLOW_DEMO_KEY') as c:
    embeddings = list(c.embed_sentences(texts))

In [21]:
import sklearn

ModuleNotFoundError: No module named 'sklearn'